In [7]:
import pandas as pd
import numpy as np

In [5]:
# --- 1. CARREGAR E CONSOLIDAR TODOS OS DADOS ANOTADOS MANUALMENTE ---

# Caminhos para todos os arquivos que você anotou
path_seed_set = '/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv'
path_cycle1 = '/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle1.csv'
path_cycle2 = '/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle2.csv'
path_lote_final = '/Users/giossaurus/Developer/leia_tcc/data/processed/anotacao_maior_final.csv'

# Carregar todos os dataframes
try:
    df_seed = pd.read_csv(path_seed_set)
    df_c1 = pd.read_csv(path_cycle1)
    df_c2 = pd.read_csv(path_cycle2)
    df_final = pd.read_csv(path_lote_final)
    print("Arquivos de anotação carregados com sucesso.")
except FileNotFoundError as e:
    print(f"Erro: Arquivo não encontrado. Verifique o nome do arquivo: {e.filename}")
    # Se der erro, interrompa ou ajuste os nomes dos arquivos conforme necessário
    raise e

# Juntar todos em um único dataframe
df_full_manual = pd.concat([df_seed, df_c1, df_c2, df_final], ignore_index=True)
df_full_manual.drop_duplicates(subset=['question_id'], inplace=True)

print(f"Total de exemplos anotados manualmente consolidados: {len(df_full_manual)}")

Arquivos de anotação carregados com sucesso.
Total de exemplos anotados manualmente consolidados: 1001


In [8]:
# --- 2. SELECIONAR UMA AMOSTRA ESTRATIFICADA DE 150 EXEMPLOS ---

# Garantir que não temos classes com menos de 2 exemplos para a estratificação
value_counts = df_full_manual['intent_choice'].value_counts()
to_keep = value_counts[value_counts >= 2].index
df_filtered = df_full_manual[df_full_manual['intent_choice'].isin(to_keep)]

# Amostragem estratificada por intenção para manter a proporção
N_SAMPLES = 150
df_sample_generation = df_filtered.groupby('intent_choice', group_keys=False).apply(
    lambda x: x.sample(int(np.ceil(len(x)/len(df_filtered) * N_SAMPLES)), random_state=42)
)

/var/folders/k4/x1x0j4qs0svb48fpvhhvkmbh0000gn/T/ipykernel_80538/1188554261.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample_generation = df_filtered.groupby('intent_choice', group_keys=False).apply(


In [12]:
# --- 3. SALVAR O ARQUIVO FINAL PARA O TRABALHO DE AUTORIA ---

# Manter apenas as colunas úteis e adicionar a coluna para sua escrita
df_to_author = df_sample_generation[['question_id', 'question', 'intent_choice', 'disciplina']].copy()
df_to_author['resposta_ideal'] = '' # Adicionar coluna vazia

df_to_author.to_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/dataset_para_gerar_perguntas_respostas.csv', index=False)

print(f"\nArquivo de amostragem criado com {len(df_to_author)} exemplos.")
print("Distribuição da amostra por intenção:")
print(df_to_author['intent_choice'].value_counts())
print("\nPróximo passo: Abra o arquivo em uma planilha e comece a preencher a coluna 'resposta_ideal'.")


Arquivo de amostragem criado com 152 exemplos.
Distribuição da amostra por intenção:
intent_choice
Análise de Exemplo    60
Procedimental         42
Conceitual            40
Comparativo           10
Name: count, dtype: int64

Próximo passo: Abra o arquivo em uma planilha e comece a preencher a coluna 'resposta_ideal'.
